In [1]:
import tensorflow as tf

In [2]:
tf.matmul(tf.Variable(shape=(2, 3), initial_value=tf.random.normal(shape=(2, 3))),tf.Variable(shape=(3, 2), initial_value=tf.random.normal(shape=(3, 2))))

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-0.3150785 ,  0.16658711],
       [-2.0204403 ,  0.32159233]], dtype=float32)>

In [3]:
class Mymodel(tf.keras.Model):
    
    def __init__(self, units):
        
        super(Mymodel, self).__init__()
        
        self.W_xh = tf.Variable(shape=(2, units), initial_value=tf.random.normal(shape=(2, units)))
        self.b_h = tf.Variable(shape=(units, ), initial_value=tf.random.normal(shape=(units, )))
        
        self.W_hy = tf.Variable(shape=(units, 1), initial_value=tf.random.normal(shape=(units, 1)))
        self.b_y = tf.Variable(shape=(1, ), initial_value=tf.random.normal(shape=(1, )))
        
    def call(self, x):
        x = tf.expand_dims(x, axis=0)
        # h_: (1, units)
        h_ = tf.matmul(x, self.W_xh) + self.b_h
        h = tf.nn.tanh(h_)
        # y: (1, 1)
        y = tf.matmul(h, self.W_hy) + self.b_y
        y = tf.squeeze(y, axis=1)
        return y

In [4]:
optimizer = tf.keras.optimizers.Adam()

In [5]:
model_ = Mymodel(20)

In [6]:
def train_one_step(x, y):
    
    with tf.GradientTape() as tape:
        y_pre = model_(x)
        loss = tf.square(y - y_pre)
    
    grads = tape.gradient(loss, model_.trainable_variables)
    optimizer.apply_gradients(zip(grads, model_.trainable_variables))
    return loss

In [7]:
import numpy as np

In [8]:
data_x = tf.constant([[1.0, 1.0], [-1.0, 1.0], [1.0, -1.0], [-1.0, -1.0]])
data_y = tf.constant([1.0, -1.0, -1.0, 1.0])

In [9]:
data_y

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 1., -1., -1.,  1.], dtype=float32)>

In [10]:
data_x

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[ 1.,  1.],
       [-1.,  1.],
       [ 1., -1.],
       [-1., -1.]], dtype=float32)>

In [11]:
data = tf.data.Dataset.from_tensor_slices((data_x, data_y))

In [12]:
EPOCHS = 1000

In [13]:
%%time
for epoch in range(EPOCHS):
    
    epoch_loss = 0
    
    for x, y in data:
        tmp_loss = train_one_step(x, y)
        epoch_loss += tmp_loss
    
    if epoch % 100 == 0:
        print("epoch: {}, loss: {}".format(epoch, epoch_loss))

epoch: 0, loss: [22.172821]
epoch: 100, loss: [0.68087184]
epoch: 200, loss: [0.02233239]
epoch: 300, loss: [0.00015291]
epoch: 400, loss: [1.6550109e-07]
epoch: 500, loss: [2.421281e-10]
epoch: 600, loss: [1.33333344e-11]
epoch: 700, loss: [3.474554e-12]
epoch: 800, loss: [2.135181e-12]
epoch: 900, loss: [9.450218e-13]
CPU times: user 27.6 s, sys: 260 ms, total: 27.9 s
Wall time: 27.5 s
